# Tutorial 1: reading and preparing data for an HMP model

In order to fit an HMP model we need neural time-serie data (typically EEG) along with the information of the Reaction Time (RT). The data will then be parsed into matrices of values with shape ```epochs X channels X samples X participants``` where samples ranges from the minimum RT to the maximum RT in the data (therefore consider those boundaries carefully). The data of each trial will be cut before stimulus onset and after the recording of a response both because 1) we don't usually consider events outside of this window and 2) because the data can be too large if we just take all the samples in a given window. This tutorial shows you several way to going from recorded EEG data to the expected format

In this tutorial we will show you how to read data from :

1. Epoched data (recommended)
3. Raw (preprocessed) EEG data in *.bdf* or *.fif* format
4. Data already transformed to PC space (e.g. previous applications of HMP) and example of how to adapt to the expected format

All these methods assume that the data has been preprocessed (e.g. see method section of [this paper](https://psyarxiv.com/nmg6w/) for an example of preprocessing for HMP), if you don't know where to start for preprocessing EEG, the [MNE](https://mne.tools/stable/index.html) is full of tutorials on EEG data.

As other sources of example in preprocessing you can take a look at [an explained example](./preprocessing_example/Template_preprocessing.ipynb) for a full ```mne``` preprocessing pipeline or [here](./sample_data/eeg/0025.ipynb) for an adaptation of matlab data to epoch ready for HMP

# General data format

## 1. Epoched data (recommended)

Starting from epoched data is the recommended method as the internal functions in hsmm_mvpy to epoch data (see next section) might not be adapted to any kind of experiment. 

An HMP model works on a single trial basis. A trial is defined as the onset of a stimulus and the time at which the response was provided (reation time, RT). Now, to restric the analyzed data to this duration we need to pass the information of the trials RT along the time-series we are using (e.g. EEG/MEG channels). The easiest way of doing this is to rely on the triggers that were co-registered with the data and that signals useful event such as which stimulus was presented and wich answer was given. In MNE this is done by using the metadata (see MNE's [tutorial](https://mne.tools/dev/auto_tutorials/epochs/30_epochs_metadata.htmlhttps://mne.tools/dev/auto_tutorials/epochs/30_epochs_metadata.html)). If your data is already epoched and the co-registered data or conditions that you want to use in the HMP models are already stored in the epoched MNE data then loading data is very easy.

Here is an example

### 1.2 Example with metadata

For this section we will rely on data that can be found in the ```sample_data/``` folder. This data was collected for [this](https://www.sciencedirect.com/science/article/pii/S1053811914002249) paper and fitted using HMP in [this](https://link.springer.com/article/10.1007/s42113-021-00105-2) paper. We downloaded and preprocessed (see preprocessing notebooks in ```sample_data/eeg/```) the data that can be found on the [OSF project](https://osf.io/pd3kf/) of the former paper.

__Imprtant note__ Given that we are using the RT as a cutting point selecting appropriate trimming is very important. For example a typical perceptual decision making is unlikely to be made with RT shorter than 300 ms nor RT longer than 3 seconds. Therefore the next function uses ```lower_limit_RT=0.3``` and ```upper_limit_RT=3``` to apply these criteria.

In [26]:
#Development only
import sys
sys.path.insert(0, "/home/gweindel/owncloud/projects/RUGUU/hsmm-mvpy/src")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
import os 
import hsmm_mvpy as hmp
sfreq = 100 #at what sampling rate we want the data, downsampling to 100Hz is computationally less intensive for hmp instances

eeg_data_path = os.path.join('sample_data/eeg/')#Declaring path where the eeg data is
subj_names = [x.split('.')[0] for x in os.listdir(eeg_data_path) if 'fif' in x]#Extracting subject name based on the file name
subj_files = [eeg_data_path+x+'.fif' for x in subj_names]#Create a list of files that can be read from the folder

epoch_data = hmp.utils.read_mne_data(subj_files, epoched=True, sfreq=sfreq,
                            lower_limit_RT=0.3, upper_limit_RT=3, rt_col='RT', scale=1000,
                            verbose=True, subj_idx=subj_names)#Turning verbose off for the documentation but it is recommended to leave it on as some output from MNE might be useful

Processing participant sample_data/eeg/processed_0023_2023_epo.fif's epoched eeg
Reading /home/gweindel/owncloud/projects/RUGUU/hsmm-mvpy/tutorials/sample_data/eeg/processed_0023_2023_epo.fif ...
    Found the data of interest:
        t =    -250.00 ...    2990.00 ms
        0 CTF compensation matrices available
Adding metadata with 6 columns
193 matching events found
No baseline correction applied
0 projection items activated
Applying reaction time trim to keep RTs between 0.3 and 3 seconds
191 RTs kept of 193 clean epochs
191 trials were retained for participant sample_data/eeg/processed_0023_2023_epo.fif
End sampling frequency is 100 Hz
Processing participant sample_data/eeg/processed_0022_2023_epo.fif's epoched eeg
Reading /home/gweindel/owncloud/projects/RUGUU/hsmm-mvpy/tutorials/sample_data/eeg/processed_0022_2023_epo.fif ...
    Found the data of interest:
        t =    -250.00 ...    2990.00 ms
        0 CTF compensation matrices available
Adding metadata with 6 columns
182 m

```hmp.utils.read_mne_data``` automatically reads the epoched data and converts it to an xarray dataset. Any metadata that has been saved along zith the epoched data is also kept (hence avoid too large metadata and only keep the important metadata).

In [88]:
epoch_data

<xarray.Dataset>
Dimensions:      (epochs: 200, electrodes: 30, samples: 211, participant: 4)
Coordinates:
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * electrodes   (electrodes) <U3 'FP1' 'FP2' 'AFz' 'F7' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 ... 204 205 206 207 208 209 210
  * participant  (participant) <U23 'processed_0023_2023_epo' ... 'processed_...
Data variables:
    data         (participant, epochs, electrodes, samples) float64 -0.0591 ....
    stim         (participant, epochs) float64 1.0 1.0 2.0 2.0 ... 2.0 nan 2.0
    resp         (participant, epochs) object 'resp_left' ... 'resp_left'
    RT           (participant, epochs) float64 1.576e+03 596.0 ... nan 1.02e+03
    cue          (participant, epochs) object 'AC' 'SP' 'SP' ... 'SP' nan 'AC'
    movement     (participant, epochs) object 'stim_left' ... 'stim_right'
    trigger      (participant, epochs) object 'AC/stim_left/resp_left' ... 'A...
Attributes:
    sfreq:    100.0
    offset:   0

And we can now save the data for use in the next tutorials:

In [89]:
epoch_data.to_netcdf('sample_data/sample_data.nc')

### 1.2 Example without metadata

If you haven't stored the metadata along with the epoched data you can always provide a list of RTs and condition to parse the data, However you need to be sure that the order of the trials from which you take the info is the same as the EEG data. Here an example with the same data as previously:

In [102]:
import pandas as pd#using pandas to generate the metadata
import xarray as xr#merging the datasets of the individual participants
import numpy as np 
import os 
import hsmm_mvpy as hmp
sfreq = 100 #at what sampling rate we want the data, downsampling to 100Hz is computationally less intensive for hmp instances


behavioral_data_path = os.path.join('sample_data/behavior/')#Declaring path where the eeg data is
subj_names_EEG = [x.split('.')[0] for x in os.listdir(eeg_data_path) if 'fif' in x]#Extracting subject name based on the file name
subj_eeg_files = [eeg_data_path+x+'.fif' for x in subj_names_EEG]#Create a list of files that can be read from the folder
subj_names = [behavioral_data_path+x.split('.')[0] for x in os.listdir(behavioral_data_path) if 'csv' in x]#Extracting subject name based on the file name

data_group = []#Where to store participants data

for participant in zip(subj_names,subj_eeg_files):#for loop for each participant in the experiment
    print(participant)#keeping track
    #First we read the behavioral data, the name of the colums is at the end of the next line and will depend on your experiment
    behavior = pd.read_csv(os.path.join('%s.csv'%participant[0]), sep=';')[['stim', 'resp', 'RT','cue', 'movement',]]
    #Next we need to store the infos of the RT and experimental conditions for each recorded epochs
    #(optional) we can rename the varaiable that are numerically coded, this does make your code more transparent for others
    behavior['movement'] = behavior.apply(lambda row: 'stim_left' if row['movement'] == -1 else 
                                      ('stim_right' if row['movement'] == 1 else np.nan), axis=1)
    behavior['resp'] = behavior.apply(lambda row: 'resp_left' if row['resp'] == 1 else 
                                      ('resp_right' if row['resp'] == 2 else np.nan ), axis=1)
    #Merging together the exeperimental conditions info to have the format condition/stimulus/response
    behavior['trigger'] = behavior['cue'] + '/' +  behavior['movement'] + '/' +  behavior['resp']
    
    #Next if you want some RTs not to be analyzed (e.g. less than 300 ms because unrealistically fast) you should do this by zeroing out outliers RTs
    behavior['RT'] = behavior.apply(lambda row: 0 if row['RT'] < 300 else (
                0 if row['RT'] > 3000 else row['RT']), axis = 1)
    
    #Then we read the associated EEG data 
    data_group.append(hmp.utils.read_mne_data(participant[1], epoched=True, sfreq=sfreq,
                     lower_limit_RT=0.3, upper_limit_RT=3,\
                     metadata = behavior, rt_col='RT', scale=1000, verbose=False, subj_idx=participant[1].split('/')[-1]))#Turning verbose off for the documentation but it is recommended to leave it on as some output from MNE might be useful    #And use the integrated function in hmp to cut off the epochs to the duration of RTs and declare at which sampling frequency the data is

#Store all this into an xarray file with new 'participant' dimentsion
epoch_data = xr.concat(data_group, pd.Series(subj_names, name="participant"),
                  fill_value={'event':'', 'data':np.nan})

('sample_data/behavior/0025-cnv-sat3_ET', 'sample_data/eeg/processed_0023_2023_epo.fif')
Processing participant sample_data/eeg/processed_0023_2023_epo.fif's epoched eeg
191 trials were retained for participant sample_data/eeg/processed_0023_2023_epo.fif
('sample_data/behavior/0024-cnv-sat3_ET', 'sample_data/eeg/processed_0022_2023_epo.fif')
Processing participant sample_data/eeg/processed_0022_2023_epo.fif's epoched eeg
182 trials were retained for participant sample_data/eeg/processed_0022_2023_epo.fif
('sample_data/behavior/0022-cnv-sat3_ET', 'sample_data/eeg/processed_0024_2023_epo.fif')
Processing participant sample_data/eeg/processed_0024_2023_epo.fif's epoched eeg
171 trials were retained for participant sample_data/eeg/processed_0024_2023_epo.fif
('sample_data/behavior/0023-cnv-sat3_ET', 'sample_data/eeg/processed_0025_2023_epo.fif')
Processing participant sample_data/eeg/processed_0025_2023_epo.fif's epoched eeg
178 trials were retained for participant sample_data/eeg/processe

And this leads to exactly the same data

In [103]:
epoch_data

<xarray.Dataset>
Dimensions:      (epochs: 200, electrodes: 30, samples: 211, participant: 4)
Coordinates:
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * electrodes   (electrodes) <U3 'FP1' 'FP2' 'AFz' 'F7' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 ... 204 205 206 207 208 209 210
  * participant  (participant) object 'sample_data/behavior/0025-cnv-sat3_ET'...
Data variables:
    data         (participant, epochs, electrodes, samples) float64 -0.0591 ....
    stim         (participant, epochs) float64 1.0 1.0 2.0 2.0 ... 2.0 nan 2.0
    resp         (participant, epochs) object 'resp_left' ... 'resp_left'
    RT           (participant, epochs) float64 1.576e+03 596.0 ... nan 1.02e+03
    cue          (participant, epochs) object 'AC' 'SP' 'SP' ... 'SP' nan 'AC'
    movement     (participant, epochs) object 'stim_left' ... 'stim_right'
    trigger      (participant, epochs) object 'AC/stim_left/resp_left' ... 'A...
Attributes:
    sfreq:    100.0
    offset:   0

## 1.3 Example with other types of stored epoched data

Lastly perhaps you have your epoched data saved in another format for example a csv file. In this case things are a bit less automatic and you need to carefully look at the data you saved. If we take the same data as previously this would be how you go from both the EEG and behavior saved as csv files

In [106]:
import os 
import numpy as np 
import pandas as pd
import xarray as xr
import hsmm_mvpy as hmp

eeg_data_path = os.path.join('sample_data/eeg/')#Declaring path where the eeg data is
subj_names = [x.split('.')[0] for x in os.listdir(eeg_data_path) if 'csv' in x]#Extracting subject name based on the file name
print(subj_names)

behavioral_data_path = os.path.join('sample_data/behavior/')#Declaring path where the eeg data is

data_group = []#Where to store participants data

for participant in subj_names:#for loop for each participant in the experiment
    print(participant)#keeping track
    #First we read the behavioral data, the name of the colums is at the end of the next line and will depend on your experiment
    behavior = pd.read_csv(os.path.join(behavioral_data_path+'%s-cnv-sat3_ET.csv'%participant), sep=';')[['stim', 'resp', 'RT','cue', 'movement',]]
    #Next we need to store the infos of the RT and experimental conditions for each recorded epochs
    #(optional) we can rename the varaiable that are numerically coded, this does make your code more transparent for others
    behavior['movement'] = behavior.apply(lambda row: 'stim_left' if row['movement'] == -1 else 
                                      ('stim_right' if row['movement'] == 1 else np.nan), axis=1)
    behavior['resp'] = behavior.apply(lambda row: 'resp_left' if row['resp'] == 1 else 
                                      ('resp_right' if row['resp'] == 2 else np.nan ), axis=1)
    #Merging together the exeperimental conditions info to have the format condition/stimulus/response
    behavior['trigger'] = behavior['cue'] + '/' +  behavior['movement'] + '/' +  behavior['resp']
    
    #Next if you want some RTs not to be analyzed (e.g. less than 300 ms because unrealistically fast) you should do this by zeroing out outliers RTs
    behavior['RT'] = behavior.apply(lambda row: 0 if row['RT'] < 300 else (
                0 if row['RT'] > 3000 else row['RT']), axis = 1)
    
    #Then we read the associated EEG data 
    data = pd.read_csv(eeg_data_path+participant+'.csv', index_col=0).reset_index(drop=True)
    #Drop any column that is not time, epoch or electrode name
    data = data.drop(columns='condition')
    #For this specific data we need to rename FP1 and FP2 to align to montage convention
    data = data.rename(columns={'FP1':'Fp1', 'FP2':'Fp2'})
    #And use the integrated function in hmp to cut off the epochs to the duration of RTs and declare at which sampling frequency the data is
    data_group.append(hmp.utils.parsing_epoched_eeg(data=data, rts=behavior['RT'], conditions=behavior['trigger'],  sfreq=100))

#Store all this into an xarray file with new 'participant' dimentsion
epoch_data = xr.concat(data_group, pd.Series(subj_names, name="participant"),
                  fill_value={'event':'', 'data':np.nan})

['0025', '0024', '0023', '0022']
0025
Totaling 178 valid trials
0024
Totaling 183 valid trials
0023
Totaling 193 valid trials
0022
Totaling 182 valid trials


In [103]:
epoch_data

<xarray.Dataset>
Dimensions:      (epochs: 200, electrodes: 30, samples: 211, participant: 4)
Coordinates:
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * electrodes   (electrodes) <U3 'FP1' 'FP2' 'AFz' 'F7' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 ... 204 205 206 207 208 209 210
  * participant  (participant) object 'sample_data/behavior/0025-cnv-sat3_ET'...
Data variables:
    data         (participant, epochs, electrodes, samples) float64 -0.0591 ....
    stim         (participant, epochs) float64 1.0 1.0 2.0 2.0 ... 2.0 nan 2.0
    resp         (participant, epochs) object 'resp_left' ... 'resp_left'
    RT           (participant, epochs) float64 1.576e+03 596.0 ... nan 1.02e+03
    cue          (participant, epochs) object 'AC' 'SP' 'SP' ... 'SP' nan 'AC'
    movement     (participant, epochs) object 'stim_left' ... 'stim_right'
    trigger      (participant, epochs) object 'AC/stim_left/resp_left' ... 'A...
Attributes:
    sfreq:    100.0
    offset:   0

# 2. Reading raw data

For the following I give an example using (yet not public) preprocessed EEG data that has not yet been epoched. This section is useful if you:
- Do not have the behavioral data to use RTs and associated conditions
- Haven't epoched the data or just want to take a quick look at HMP models with raw data laying around

__Nota bene__: the data is assumed to be preprocessed for artifacts and eye-related activity (see an example here: https://github.com/GWeindel/hsmm_mvpy/tree/main/tutorials/preprocessing_example)

First in order for the HMP to run on each trial we have to declare the stimulus triggers as well as the response triggers which defines a trial. 
In the case of this data, the stimulus trigger is a 3 digit code where the first digit codes for the condition (speed or accuracy), the second for the side of the stimulus correct answer and the third one to the stimulus difficulty level. The response trigger codes either left responses or right responses. These triggers will then be used as info for the specific condition of the trial as well as its reaction time (i.e. the time between the stimulus trigger onset and the response trigger onset)

(see Chapter 5 of https://thesiscommons.org/342zp if you want more details on the data and the design)

In [119]:
stimulus_id = {'accuracy/left/1':211,'accuracy/left/2':212,'accuracy/left/3':213,
           'accuracy/right/1':221,'accuracy/right/2':222,'accuracy/right/3':223,
            'speed/left/1':111,'speed/left/2':112,'speed/left/3':113, #conditions in the experiment
           'speed/right/1':121,'speed/right/2':122,'speed/right/3':123} # used for segmentation
resp_id = {'r_left':100,'r_right':200}#Response events

Then we specify wich files we want to analyse, if for example you have all your participant dat inside one folder you can do the following :

In [120]:
import os
eeg_path = os.path.join('../../../PHD/ForceEEG_2021/processing_EEG/preprocessed_data/')#Where the raw EEG data is relative to where you run the code on your computer
subj_names = [x.split('_')[1].split('.')[0] for x in os.listdir(eeg_path) if 'preprocessed' in x]#select files which contain the string 'preprocessed'
subj_names = subj_names[:2]#Take the two first participant for the illustration
subj_files = [eeg_path+ 'preprocessed_'+x+'_raw.fif' for x in subj_names]#Create a list of files that can be read from the folder

Next we can directly use the ```subj_file``` list and feed it to the ```hmp.utils.read_mne_data``` but first you have to decide on a number of parameters regarding EEG processing, see the following helper of the function

In [121]:
from hsmm_mvpy import utils
utils.read_mne_data?

Signature:
utils.read_mne_data(
    pfiles,
    event_id=None,
    resp_id=None,
    epoched=False,
    sfreq=None,
    subj_idx=None,
    metadata=None,
    events_provided=None,
    rt_col='rt',
    rts=None,
    verbose=True,
    tmin=-0.2,
    tmax=5,
    offset_after_resp=0,
    high_pass=0.5,
    low_pass=None,
    pick_channels='eeg',
    baseline=(None, 0),
    upper_limit_RT=5,
    lower_limit_RT=0.001,
    reject_threshold=None,
    scale=1,
)
Docstring:
Reads EEG/MEG data format (.fif or .bdf) using MNE's integrated function .

Notes: 
- Only EEG or MEG data are selected (other channel types are discarded)
- All times are expressed on the second scale.
- If multiple files in pfiles the data of the group is read and seqentially processed.
- For non epoched data: Reaction Times are only computed if response trigger is in the epoch window (determined by tmin and tmax)

Procedure:
if data not already epoched:
    0.1) the data is filtered with filters specified in low_pass and h

This is then an example call to the function based on previous _hmp_ applications

In [126]:
import hsmm_mvpy as hmp
sfreq = 100 #at what sampling rate we want the data, downsampling to 100Hz is computationally less intensive for hmp instances
tmin, tmax = -.25, 2 #window size for the epochs, from 250ms before the stimulus up to 2 seconds after, data will be baseline corrected from tmin to 0
high_pass = .5 #High pass filtering to be applied, useful to avoid slow drifts, parameters of the filters are defined automatically by MNE
lower_limit_RT, upper_limit_RT = .2, 2 #lower and upper limit for the RTs all values outside of this range are discarded

mne_data = hmp.utils.read_mne_data(subj_files, event_id=stimulus_id, resp_id=resp_id, 
                            sfreq=sfreq, subj_idx=subj_names, tmin=tmin, tmax=tmax, 
                            high_pass=high_pass, lower_limit_RT=lower_limit_RT, upper_limit_RT=upper_limit_RT, 
                            verbose=False)#Turning verbose off for the documentation but it is recommended to leave it on as some output from MNE might be useful

Processing participant ../../../PHD/ForceEEG_2021/processing_EEG/preprocessed_data/preprocessed_S10_raw.fif's continuous eeg
Correcting event values as trigger channel has offset [768]
Reading 0 ... 4814847  =      0.000 ...  4701.999 secs...
Downsampling to 100 Hz
2290 trials were retained for participant ../../../PHD/ForceEEG_2021/processing_EEG/preprocessed_data/preprocessed_S10_raw.fif
Processing participant ../../../PHD/ForceEEG_2021/processing_EEG/preprocessed_data/preprocessed_S18_raw.fif's continuous eeg
Correcting event values as trigger channel has offset [768]
Reading 0 ... 5106687  =      0.000 ...  4986.999 secs...
Downsampling to 100 Hz


ValueError: metadata must have the same number of rows (2294) as events (2448)

This is then the data ready to be used with hmp instances

In [ ]:
mne_data

# 3. Data already transformed to PC space (e.g. previous applications of HMP MVPA)


This part is addressed mainly for researchers wanting to replicate previous findings with matlab with data that has already been turned into principal component space and contains therefore __very specific code__ that most users will probably not need. Users who have other data format as the one listed in the previous section can probably also use this code to adapt to their specific format.

For this example we will use the data from the study by Anderson, Zhang, Borst, & Walsh ([2016](https://psycnet.apa.org/doi/10.1037/rev0000030)) that you can download here: http://www.ai.rug.nl/~jpborst/modelbasedneuro/analysis_data_model.zip

__IMPORTANT__: if you want to run this part of the notebook you need to download the [data](http://www.ai.rug.nl/~jpborst/modelbasedneuro/analysis_data_model.zip) and put the files ```analysis_data_model/data/varForevents135_100.mat```  and ```analysis_data_model/analysis/HSMM_code/chanlocs.mat``` in the same folder as the notebook.

First we read the .mat data file using ```scipy```

In [ ]:
import scipy.io
import numpy as np

mat = scipy.io.loadmat('varForBumps135_100.mat')#Where the .mat file with all the data and necessary infos is
data = np.array(mat['normedscore10'])# The data in PC space
starts = np.array(mat['x'][:,0]) -1#correcting to be 0 indexed
ends = np.array(mat['y'][:,0])-1#correcting to be 0 indexed
subjects = np.array(mat['subjects'])-1 #correcting to be 0 indexed

Here we will adapt the structure of the matlab data to the expected data format

In [ ]:
### Recovering RTs, trials and participants
durations = ends - starts + 1
limits = np.cumsum(durations)
limits = np.concatenate([[0],limits])
participant, trials = np.unique(subjects, return_counts=True)


# creating data watrix with participant x trial x max duration x electrodes/comp
unstacked_data = np.tile(np.nan, (len(participant), np.max(trials), np.shape(data)[1], np.max(durations)))
conditions = np.tile(np.nan, (len(participant), np.max(trials)))
for trial in np.arange(len(durations)):
    if trial > 0 and subjects[trial] == subjects[trial-1]:
        within_trial += 1
    else:
        within_trial = 0
    unstacked_data[subjects[trial], within_trial, :,:durations[trial],] = data[limits[trial]:limits[trial+1]].T
    conditions[subjects[trial], within_trial] = mat['conds'][trial]

unstacked_data = hmp.utils.hmp_data_format(unstacked_data, conditions, sfreq=100, participants=participant)#converting to xarray
unstacked_data = unstacked_data.rename({'electrodes':'component'})#Data is already on PC space so no need to use the transform function
unstacked_data.to_netcdf('unstacked_data.nc')#saving the converted data

In [ ]:
unstacked_data

Now the data is in the expected format for HMP however we still need to recover the electrodes position

In [ ]:
channels_loc = scipy.io.loadmat('chanlocs.mat')#Channel location 
positions_dict_i =  [{str(electrode[0][0]):[-electrode[5][0][0]/1000,electrode[4][0][0]/1000,electrode[6][0][0]/1000]} 
                     for electrode in channels_loc['chanlocs'][0]]#Dictionnary with x,y,z positions
positions_dict = {}
for i in positions_dict_i:
     positions_dict.update(i)
        
from mne import channels#

montage = channels.make_dig_montage(positions_dict,coord_frame='head')
positions = np.array([x[-1][:2] for x in np.array(list(montage.get_positions()['ch_pos'].items()),dtype=object)[:,:]])
names = np.array([x[0] for x in np.array(list(montage.get_positions()['ch_pos'].items()),dtype=object)[:,:]])
montage.plot();
np.save('positions',positions)#saving the positions

Everything is now ready to be used in _hmp_

In [ ]:
stacked_data = hmp.utils.stack_data(unstacked_data.data)
init = hmp.models.hmp(stacked_data, sfreq=100, event_width=50, cpus=4)

__For data that cannot be read by MNE:__ Here is an example to run and plot a 5 event model once it is uncommented:

In [ ]:
# model = init.fit_single(5) 
# mean_event_times_selected = init.event_times(model.eventprobs.dropna('event'))
# electrodes_selected = xr.DataArray(hsmm.utils.reconstruct(model.magnitudes.T, mat['coeff10'][:,:10], 
#                             mat['latent10'][:10], np.mean(mat['data'],axis=0)))
# hsmm.visu.plot_topo_timecourse(electrodes_selected, mean_event_times_selected, positions, 
#                                init, magnify=1, figsize=(12,1),
#                                 time_step = 1000/init.sfreq,  times_to_display = np.mean(ends - starts),
#                                 ylabels='5 events model', vmin=-10, vmax=10)